# Portfolio's Building #

### Rebalancing Weights ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Optimization
from scipy.optimize import minimize

# Visualization
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from config import get_tickers
from data_downloader import get_market_data
from portfolios_toolkit import calculate_analytics
from portfolios_toolkit import markowitz_weights

In [2]:
tickers = get_tickers(mod="2.4")

tickers

In [22]:
# Import data
df_returns = pd.DataFrame()

for ticker in tickers:
    df = get_market_data(
        ticker=ticker, 
        start_date='2013-01-01', 
        end_date='2025-01-01', 
        returns=True
    )
    
    returns = df['returns'].rename(ticker)
    
    df_returns = pd.concat([df_returns, returns], axis=1)
    
    print(f'Data Ready for {ticker}')

df_returns.index.name = 'date'
df_returns.index = pd.to_datetime(df_returns.index)

In [23]:
df_returns

In [24]:
df_returns.mean()

In [25]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns.cumsum(), label=df_returns.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [26]:
# Let us Calculate the Weights
def rolling_weights(
    returns, 
    desired_returns, 
    window=252, 
    rebalance_freq=126
):

    # Lists to Store Things
    weights_list = []
    dates = []

    for i in range(window, len(returns), rebalance_freq):
        past_returns = returns.iloc[i - window:i]  # Rolling Window
        past_excepted_returns = past_returns.mean()
        past_cov_matrix = past_returns.cov()

        # Calculate Weights
        w = markowitz_weights(past_excepted_returns, past_cov_matrix, desired_returns)

        # Save weights and dates
        weights_list.append(w)
        dates.append(returns.index[i])

    # Create the DataFrame
    weights_df = pd.DataFrame(weights_list, index=dates, columns=returns.columns)

    # Expand the DataFrame
    weights_df = weights_df.reindex(returns.index, method='ffill')

    return weights_df.dropna()

In [27]:
# Create the DataFrames of Returns
df_weights = rolling_weights(df_returns, 0.001)

df_weights

In [28]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_weights, label=df_weights.columns, alpha=1)
# Config
plt.title('Weights Time Series')
plt.xlabel('Time Index')
plt.ylabel('Weights')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [29]:
# Common Index
common_index = df_returns.index.intersection(df_weights.index)  # Fechas en común
df_returns_reindex = df_returns.reindex(common_index)
df_weights = df_weights.reindex(common_index)

df_returns_reindex

In [30]:
# Create the Portfolio Returns
df_weighted_returns = df_returns_reindex * df_weights

df_weighted_returns

In [31]:
# Create the Portfolio Returns
df_returns_portfolio = df_returns.copy()

# Add the columns
df_returns_portfolio['Portfolio'] =  df_weighted_returns.sum(axis = 1)

df_returns_portfolio['Portfolio'].dropna()

In [32]:
# Time Series Graphs

df_plot = df_returns_portfolio.dropna()

plt.figure(figsize=(10, 6))
plt.plot(df_plot.cumsum(), label=df_plot.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Define different rebalancing frequencies ###

In [33]:
# Desired Return
dr = 0.001

In [34]:
# Create weights for different rebalancing frequencies
df_weights_5d = rolling_weights(df_returns, dr, rebalance_freq=5)
df_weights_21d = rolling_weights(df_returns, dr, rebalance_freq=21)
df_weights_63d = rolling_weights(df_returns, dr, rebalance_freq=63)
df_weights_126d = rolling_weights(df_returns, dr, rebalance_freq=126)
df_weights_252d = rolling_weights(df_returns, dr, rebalance_freq=252)

In [35]:
# Create the Returns
df_weighted_returns_5d = df_returns * df_weights_5d
df_weighted_returns_21d = df_returns * df_weights_21d
df_weighted_returns_63d = df_returns * df_weights_63d
df_weighted_returns_126d = df_returns * df_weights_126d
df_weighted_returns_252d = df_returns * df_weights_252d

In [36]:
# Add the columns
portfolios_df = pd.DataFrame(index = df_returns.loc['2016':].index)

portfolios_df['5d_port'] = df_weighted_returns_5d.sum(axis = 1)
portfolios_df['21d_port'] = df_weighted_returns_21d.sum(axis = 1)
portfolios_df['63d_port'] = df_weighted_returns_63d.sum(axis = 1)
portfolios_df['126d_port'] = df_weighted_returns_126d.sum(axis = 1)
portfolios_df['252d_port'] = df_weighted_returns_252d.sum(axis = 1)

portfolios_df

In [37]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(portfolios_df.cumsum().mul(100), label=portfolios_df.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns (%)')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Comparing Portfolios ###

In [38]:
# Now the table
analytics_table = calculate_analytics(portfolios_df)

analytics_table

### Constraining Weights ###

The problem is that we cannot create large portfolios (with many components) using this methodology because, sometimes, many assets will have high levels of leverage. Therefore, we can use other methods:

In [39]:
# Adding weights restrictions:
def markowitz_scipy(
    expected_returns,
    cov_matrix,
    target_return,
    max_weight=0.1
):
    # Number of observations
    n = len(expected_returns)
    
    # Objective Function: Variance
    def objective(w):
        return w.T @ cov_matrix @ w
    
    # Constraints:
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},                         # Weights must sum 1
        {'type': 'eq', 'fun': lambda w: expected_returns @ w - target_return}   # Objective Returns
    ]
    
    # Set limits for weights
    bounds = [(-max_weight, max_weight) for _ in range(n)]
    
    # Initial weights (equal weighted)
    w0 = np.ones(n) / n
    
    # Optimization (Minimize Variance)
    result = minimize(
        objective, 
        w0, 
        method='SLSQP', 
        bounds=bounds, 
        constraints=constraints
    )
    
    if result.success:
        return result.x
    else:
        raise ValueError("La optimización falló: " + result.message)

In [40]:
# Import Data
df_returns_big = pd.read_csv(r'..\additional_data\stocks_returns.csv')
df_returns_big = df_returns_big.rename(columns={'Unnamed: 0':'Date'})
df_returns_big.set_index('Date', inplace=True)
df_returns_big.index = pd.to_datetime(df_returns_big.index)
df_returns_big = df_returns_big.dropna(axis=1)

df_returns_big

In [41]:
# Components:
expected_returns = df_returns_big.loc['2015'].mean()
covariance_matrix = df_returns_big.loc['2015'].cov()

In [42]:
unrestricted_weights = markowitz_weights(expected_returns, covariance_matrix, 0.0005)
unrestricted_weights = pd.Series(unrestricted_weights, index = df_returns_big.columns)
unrestricted_weights.name = 'weights'

unrestricted_weights

In [43]:
restricted_weights = markowitz_scipy(expected_returns, covariance_matrix, 0.0005)
restricted_weights = pd.Series(restricted_weights, index = df_returns_big.columns)
restricted_weights.name = 'weights'

restricted_weights

In [44]:
unrestricted_portfolio = df_returns_big.loc['2016':] @ unrestricted_weights
unrestricted_portfolio.name = 'unrestricted_portfolio'

unrestricted_portfolio

In [45]:
restricted_portfolio = df_returns_big.loc['2016':] @ restricted_weights
restricted_portfolio.name = 'restricted_portfolio'

restricted_portfolio

In [46]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(unrestricted_portfolio.cumsum().mul(100), label='Portfolios with unlimited weights', alpha=1)
plt.plot(restricted_portfolio.cumsum().mul(100), label='Portfolios with limited weights', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns (%)')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [47]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(restricted_portfolio.cumsum().mul(100), label='Portfolios with limited weights', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns (%)')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [48]:
# Create the analystics
limit_portfolios = pd.DataFrame(index=restricted_portfolio.index)

limit_portfolios['unrestricted_portfolio'] = unrestricted_portfolio
limit_portfolios['restricted_portfolio'] = restricted_portfolio

limit_portfolios

In [49]:
calculate_analytics(limit_portfolios)